In [1]:
#| include: false

import duckdb
import pandas as pd
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

%sql CREATE OR REPLACE TABLE user_actions AS SELECT * FROM read_csv('00_data/sql/user_actions.csv', header=True, columns={'user_id': 'INT', 'order_id': 'INT', 'action': 'VARCHAR', 'time': 'TIMESTAMP'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE courier_actions AS SELECT * FROM read_csv('00_data/sql/courier_actions.csv', header=True, columns={'courier_id': 'INT', 'order_id': 'INT', 'action': 'VARCHAR', 'time': 'TIMESTAMP'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE orders AS SELECT * FROM read_csv('00_data/sql/orders.csv', header=True, columns={'order_id': 'INT', 'creation_time': 'TIMESTAMP', 'product_ids': 'INT[]'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE users AS SELECT * FROM read_csv('00_data/sql/users.csv', header=True, columns={'user_id': 'INT', 'birth_date': 'DATE', 'sex': 'VARCHAR'}, dateformat='%d/%m/%y');
%sql CREATE OR REPLACE TABLE couriers AS SELECT * FROM read_csv('00_data/sql/couriers.csv', header=True, columns={'courier_id': 'INT', 'birth_date': 'DATE', 'sex': 'VARCHAR'}, dateformat='%d/%m/%y');
%sql CREATE OR REPLACE TABLE products AS SELECT * FROM read_csv('00_data/sql/products.csv', header=True, columns={'product_id': 'INT', 'name': 'VARCHAR', 'price': 'DOUBLE'});

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


,Count
0,87


# Групування даних

## Оператор `GROUP BY`
Для групування даних в SQL використовується оператор `GROUP BY`. Він дозволяє групувати дані по одному або декільком стовпцям таблиці. При цьому вибірка даних буде містити лише унікальні значення з вказаних стовпців:

1. Спочатку в таблиці визначаються рядки, в яких у вказаному в `GROUP BY` стовпці є однакові значення.
2. Далі за цими значеннями записи об'єднуються у групи, причому у групі може бути навіть один запис.
3. Після цього над елементами цих груп, як правило, проводяться якісь операції за допомогою агрегатних функцій: наприклад, за допомогою `SUM()` обчислюється сума значень в якому-небудь стовпці в кожній групі:

```sql
SELECT column_1, SUM(column_2)
FROM table
GROUP BY column_1
```

4. Потім у результуючий запит потрапляє по одному унікальному запису зі стовпця, по якому здійснювалося групування. При цьому навпроти кожного такого значення відображається результат агрегації по відповідній групі, якщо агрегація проводилася.

Тут важливо зробити кілька уточнень:

- По-перше, групування виконується після фільтрації, тобто спочатку виконуються інструкції `WHERE` і лише потім дані групуються через `GROUP BY`.
- По-друге, до груп, що утворилися в результаті застосування `GROUP BY`, можна застосовувати відразу кілька агрегатних функцій (у тому числі до різних колонок).
- По-третє, групування можна робити відразу за новими полями, порахованими в `SELECT`: при цьому допускається використання в `GROUP BY` аліасу колонки, зазначеного в SELECT. Наступні два запити дадуть однаковий результат:

```sql
SELECT DATE(column_1) AS date, SUM(column_2)
FROM table
GROUP BY DATE(column_1)


SELECT DATE(column_1) AS date, SUM(column_2)
FROM table
GROUP BY date
```

- По-четверте, робити агрегацію після групування необов'язково. Якщо не вказувати агрегатну функцію, то запит поверне унікальні значення в стовпці, тобто той самий результат, як і оператор `DISTINCT`. Можете самостійно запустити наступний запит та переконатися:

```sql
SELECT user_id
FROM user_actions
GROUP BY user_id

SELECT DISTINCT user_id
FROM user_actions
```

До речі, питання про те, як можна відібрати унікальні значення, досвідчені аналітики люблять ставити на співбесідах своїм менш досвідченим колегам.

І нарешті, останнє важливе уточнення: під час використання групування колонки, вказаних у `SELECT`, повинні бути і `GROUP BY`, якщо вони використовуються у агрегатних функціях. Це обов'язкова умова, і якщо вона не буде виконана, база даних поверне помилку.

Наступний запит не працюватиме, оскільки в `GROUP BY` вказані не всі неагреговані колонки з блоку `SELECT`:

```sql
SELECT column_1, column_2, SUM(column_3)
FROM table
GROUP BY column_1
```

Водночас такий запит спрацює:

```sql
SELECT SUM(column_2)
FROM table
GROUP BY column_1
```

Зверніть увагу, що у цьому запиті у блоці `SELECT` немає колонки, вказаної у `GROUP BY`, тобто у зворотний бік правило не працює: якщо ми щось вказали в `GROUP BY`, це не обов'язково вказувати в `SELECT`. Інакше кажучи, виводити найменування груп необов'язково.

І ще: замість назв колонок у блоці `GROUP BY` можна використовувати номер колонки, вказаної у `SELECT`. Наприклад, наступні два запити еквівалентні:

```sql
SELECT column_1, column_2, SUM(column_3)
FROM table
GROUP BY column_1, column_2


SELECT column_1, column_2, SUM(column_3)
FROM table
GROUP BY 1, 2
```

При цьому номери колонок із `SELECT` також можна використовувати при сортуванні в операторі `ORDER BY`. Можете самі поекспериментувати із цим у наступних завданнях.

З теорією начебто розібралися, тепер приступимо до практики. Минулого разу ми вже рахували кількість кур'єрів жіночої статі та використовували для цього фільтрацію. Групування дозволить нам провести розрахунки відразу для двох статей.

:::: {.callout-note icon=false}
## Завдання
::: {#exr-sql-groupby-01}
<br>
За допомогою групування порахуйте кількість кур'єрів чоловічої та жіночої статі у таблиці `couriers`. Нову колонку з числом кур'єром назвіть `couriers_count`. Результат відсортуйте по цій колонці за зростанням.

Поля у результуючій таблиці: `sex`, `couriers_count`
:::
::::

In [2]:
#| code-fold: true
#| code-summary: "Рішення"

%%sql
SELECT sex,
       count(courier_id) as couriers_count
FROM   couriers
GROUP BY sex
ORDER BY couriers_count

,sex,couriers_count
0,female,1149
1,male,1674


Тепер давайте аналогічним чином порахуємо максимальний вік користувачів чоловічої та жіночої статі. Тільки на цей раз виведемо не повний вік, а лише кількість повних років. Для цього до результату обчислень можна застосувати вже знайому нам функцію `DATE_PART` з аргументом `'year'`.

:::: {.callout-note icon=false}
## Завдання
::: {#exr-sql-groupby-02}
<br>
Порахуйте **максимальний вік користувачів чоловічої та жіночої статі** у таблиці `users`. Вік виміряйте кількістю **повних років**. Нову колонку з віком назвіть `max_age`. Результат відсортуйте за новою колонкою за зростанням віку.

Поля у результуючій таблиці: `sex`, `max_age`
:::
::::

In [3]:
#| code-fold: true
#| code-summary: "Рішення"

%%sql
SELECT sex,
       date_part('year', max(age(birth_date))) as max_age
FROM   users
GROUP BY sex
ORDER BY max_age

,sex,max_age
0,female,40
1,male,41


Маркетологи знову звернулися до нас із завданням: цього разу вони просять провести невеликий аналіз нашої аудиторії та порахувати, скільки клієнтів певного віку користуються нашим сервісом. Давайте допоможемо нашим колегам!

:::: {.callout-note icon=false}
## Завдання
::: {#exr-sql-groupby-03}
<br>
Розбийте користувачів з таблиці `users` на групи віком (вік вимірюємо кількістю повних років) і порахуйте кількість користувачів кожного віку. Колонку з віком назвіть `age`, а колонку з кількістю користувачів `users_count`. Відсортуйте отриманий результат за зростанням по віку. Не забудемо і про тих користувачів, у яких замість віку буде перепустки, для цієї групи також підрахуємо кількість користувачів.

Поля у результуючій таблиці: `age`, `users_count`

**Пояснення:** зверніть увагу, що у цій задачі вам необхідно провести групування за новим розрахунковим полем з віком. У цій ситуації важливо пам'ятати, що колонки, вказані в `SELECT`, повинні знаходитись і в `GROUP BY` (якщо вони не використовуються в агрегатних функціях). При цьому `GROUP BY` допускається використання аліасу колонки, зазначеного в блоці `SELECT`, тобто повторно проводити обчислення у `GROUP BY` не обов'язково.
:::
::::

In [4]:
#| code-fold: true
#| code-summary: "Рішення"

%%sql
SELECT date_part('year', age(birth_date)) as age,
       count(user_id) as users_count
FROM   users
GROUP BY age
ORDER BY age
LIMIT 10

,age,users_count
0,NaN,50
1,19.0,1
2,20.0,3
3,21.0,6
4,22.0,30
5,23.0,86
6,24.0,222
7,25.0,469
8,26.0,916
9,27.0,1497


Ви могли помітити, що результат попереднього запиту для однієї з груп повернув пусте значення віку. Ми знову зіткнулися з значеннями `NULL` — цього разу в колонці `birth_date`. Давайте позбавимося їх перед групуванням і заодно зробимо наш аналіз ще більш детальним: додамо до групування стать користувачів.

:::: {.callout-note icon=false}
## Завдання
::: {#exr-sql-groupby-04}
<br>
Знову розбийте користувачів із таблиці `users` на групи за віком (вік вимірюємо кількістю повних років), тільки тепер додайте до групування стать користувача. В результаті в кожній віковій групі має з'явитися ще по дві підгрупи зі статтю. У кожній такій підгрупі порахуйте кількість користувачів.

Усі значення `NULL` у колонці `birth_date` заздалегідь відфільтруйте за допомогою `WHERE`. Колонку з віком назвіть ag`e, а колонку з числом користувачів - `users_count`, ім'я колонки зі статтю залиште без змін. Відсортуйте отриману таблицю спочатку по колонці з віком по зростанню, потім по колонці з підлогою теж по зростанню.

Поля у результуючій таблиці: `age`, `sex`, `users_count`
:::
::::

In [5]:
#| code-fold: true
#| code-summary: "Рішення"

%%sql
SELECT date_part('year', age(birth_date)) as age,
       sex,
       count(user_id) as users_count
FROM   users
WHERE  birth_date is not null
GROUP BY age, sex
ORDER BY age, sex
LIMIT 10

,age,sex,users_count
0,19,male,1
1,20,female,1
2,20,male,2
3,21,female,3
4,21,male,3
5,22,female,14
6,22,male,16
7,23,female,41
8,23,male,45
9,24,female,119


А тепер, використовуючи наші знання про групування, давайте порахуємо скільки замовлень було зроблено і скільки скасовано в кожному окремому місяці. На цей раз для роботи з датами будемо використовувати не `DATE_PART`, а нову функцію `DATE_TRUNC`.

Функція `DATE_TRUNC` використовується усічення дат і часу, тобто вона працює аналогічно округленню `ROUND`, тільки для типів даних `TIMESTAMP` та `INTERVAL`.

Синтаксис у неї такий самий, як і у `DATE_PART`:

```sql
SELECT DATE_TRUNC(part, column)
```

На місці `part` у лапках вказується, до якої точності слід обрізати передане значення часу: `'year'`, `'month'`, `'day'`, `'hour`' тощо.

Отримані значення мають тип `TIMESTAMP` або `INTERVAL`, а всі «частини» вихідного значення, менш значущі, ніж задана «частина», прирівнюються до нуля (або одиниці, якщо це номер дня або місяця):

```sql
SELECT DATE_TRUNC('month', TIMESTAMP '2022-01-12 08:55:30')

Результат:
2022-01-01 00:00

SELECT DATE_TRUNC('day', TIMESTAMP '2022-01-12 08:55:30')

Результат:
2022-01-12 00:00	

SELECT DATE_TRUNC('hour', TIMESTAMP '2022-01-12 08:55:30')

Результат:
2022-01-12 08:00
```

::: {.callout-note}
Докладніше з функцією `DATE_TRUNC` можна ознайомитись [тут](https://duckdb.org/docs/sql/functions/date.html).
:::

:::: {.callout-note icon=false}
## Завдання
::: {#exr-sql-groupby-05}
<br>
Використовуючи функцію `DATE_TRUNC`, порахуйте, скільки замовлень було зроблено та скільки було скасовано кожного місяця. Розрахунки проводьте за таблицею `user_actions`. Колонку з усіченою датою назвіть `month`, колонку з кількістю замовлень - `orders_count`. Результат відсортуйте спочатку за місяцями – за зростанням, потім за типом дії – також за зростанням.

Поля в результуючій таблиці: `month`, `action`, `orders_count`
:::
::::

In [6]:
#| code-fold: true
#| code-summary: "Рішення"

%%sql
SELECT date_trunc('month', time) as month,
       action,
       count(order_id) as orders_count
FROM   user_actions
GROUP BY month, action
ORDER BY month, action

,month,action,orders_count
0,2022-08-01,cancel_order,967
1,2022-08-01,create_order,18799
2,2022-09-01,cancel_order,2012
3,2022-09-01,create_order,40796


У @exr-sql-agg-09 ми навчилися працювати з функцією `array_length` і навіть порахували з її допомогою кількість товарів у кожному замовленні. Давайте для кожного розміру замовлення, яке зустрічається у даних, порахуємо загальну кількість замовлень такого розміру.

:::: {.callout-note icon=false}
## Завдання
::: {#exr-sql-groupby-06}
<br>
Порахуйте кількість товарів у кожному замовленні з таблиці `orders`, застосуйте до цих значень групування та порахуйте кількість замовлень у кожній групі. Виведіть дві колонки: кількість товарів у замовленні та кількість замовлень з такою кількістю. Колонки назвіть відповідно `order_size` та `orders_count`. Результат відсортуйте за зростанням кількості товарів у замовленні.

Поля у результуючій таблиці: `order_size`, `orders_count`
:::
::::

In [7]:
#| code-fold: true
#| code-summary: "Рішення"

%%sql
SELECT array_length(product_ids, 1) as order_size,
       count(order_id) as orders_count
FROM   orders
GROUP BY order_size
ORDER BY order_size

,order_size,orders_count
0,1,3333
1,2,11851
2,3,17856
3,4,15057
4,5,8065
5,6,2774
6,7,583
7,8,71
8,9,5


## Фільтрація після групування: `HAVING`

`HAVING` використовується для фільтрації даних після групування. Він працює аналогічно `WHERE`, тільки використовується після `GROUP BY` і фільтрує вже групи, а не окремі записи.

Але треба обговорити одну важливу особливість роботи з `HAVING`: у цьому блоці не можна вказувати аліаси розрахункових полів із блоку `SELECT`. Справа в тому, що відповідно до порядку виконання запиту оператор `SELECT` виконується після оператора `HAVING`. Тому агрегацію необхідно також вказувати і `HAVING`.

Наприклад, такий запит не спрацює:

```sql
SELECT column_1, SUM(column_2) AS new_column
FROM table
GROUP BY column_1
HAVING new_column = 10
```

А такий спрацює:

```sql
SELECT column_1, SUM(column_2) AS new_column
FROM table
GROUP BY column_1
HAVING SUM(column_2) = 10
```

:::: {.callout-note icon=false}
## Завдання
::: {#exr-sql-groupby-07}
<br>
Доповніть попередній запит оператором `HAVING` та відберіть лише ті розміри замовлень, загальна кількість яких **перевищує 5000**. Знову виведіть дві колонки: кількість товарів у замовленні та кількість замовлень з такою кількістю. Колонки назвіть відповідно `order_size` та `orders_count`. Результат відсортуйте за зростанням кількості товарів у замовленні.

Поля у результуючій таблиці: `order_size`, `orders_count`
:::
::::

In [8]:
#| code-fold: true
#| code-summary: "Рішення"

%%sql
SELECT array_length(product_ids, 1) as order_size,
       count(order_id) as orders_count
FROM   orders
GROUP BY order_size
HAVING count(order_id) > 5000
ORDER BY order_size

,order_size,orders_count
0,2,11851
1,3,17856
2,4,15057
3,5,8065
